In [1]:
import yfinance as yf
from datetime import date, datetime
import pandas as pd 
import numpy as np

In [2]:
start_date = date(2000, 2, 1)
end_date = date(2021, 3, 12)

tickerSymbol = '^GSPC'
tickerData = yf.Ticker(tickerSymbol)

df = tickerData.history(period='1d', start=start_date, end=end_date).dropna()
df.drop(['Stock Splits', 'Dividends'], axis='columns', inplace=True)
df = pd.DataFrame(data = df)
df.index = pd.to_datetime(df.index)
# df["Date"] = df.index

df_copy = tickerData.history(period='1d', start=start_date, end=end_date).dropna()

df_copy.head(3)

,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
2000-01-31,1360.16,1394.48,1350.14,1394.46,993800000,0,0
2000-02-01,1394.46,1412.49,1384.79,1409.28,981000000,0,0
2000-02-02,1409.28,1420.61,1403.49,1409.12,1038600000,0,0


In [12]:
def SMA(data,n):
    
    #додаємо NaN на початку
	sma_list=[np.nan for k in range(n-1)]
    
	for i in range(len(data)-n+1):
		sum_n = 0
		for j in range(n):
			sum_n = sum_n + data[i+j]
		sma=sum_n/n
		sma_list.append(round(sma, 2))
        
	return sma_list

def EMA(data, n, a=0):
	ema_list=[data[0]]
	if a==0:
		a=2/(n+1)

	for i in range (1,len(data)):
		ema = 0
		ema = (1-a)*ema_list[i-1]+a*data[i]
		ema_list.append(ema)

	return ema_list

In [7]:
def ERI(data_close, data_high, data_low, n):
    MA = EMA(data_close, n)
#     print(MA)
    BuP, BeP = [], []
    for i in range(len(data_close)):
        BuP.append(data_high[i] - MA[i])
        BeP.append(data_low[i] - MA[i])
    return BuP, BeP# Elder-Ray Index

# Strategy ERI

In [44]:
def strategy_ERI(P_close, P_high, P_low, n=13):
    BuP_ = ERI(P_close, P_high, P_low, n)[0]
    BeP_ = ERI(P_close, P_high, P_low, n)[1]
#     print(BuP_[1:10])
#     print(BeP_[1:10])
    
    s=[np.nan]
    for i in range(1, len(BuP_)):
        if ((BeP_[i]>0)&(BeP_[i-1]<0)):
            s.append(1)
        elif ((BuP_[i]<0)&(BuP_[i-1]>0)):
#             print(BuP_[i])
#             print(BuP_[i-1])
#             print("")
#             print(BeP_[i])
#             print(BeP_[i-1])
#             print("")
            s.append(-1)
        else:
            s.append(0)
            
    return s

In [9]:
def DI(data, n, MA_type):
    if MA_type == "EMA":
        MA = EMA(data, n)
    elif MA_type == "LWMA":
        MA = LWMA(data, n)
    else:
        MA = SMA(data, n)
    
    di_list = []
    for i in range(len(MA)):
        if (np.isnan(MA[i])):
            di_list.append(np.nan)
        else:
            di_list.append((data[i]-MA[i])/(100*MA[i]))
        
    return di_list

# Strategy DI

In [21]:
def strategy_DI(P_close, n):
    DI_ = DI(P_close, n, MA_type="SMA")
    
    s=[np.nan]
    for i in range(1,len(DI_)):
        if np.isnan(DI_[i]):
            s.append(np.nan)
        elif ((DI_[i]>0)&(DI_[i-1]<0)):
            s.append(1)
        elif ((DI_[i]<0)&(DI_[i-1]>0)):
            s.append(-1)
        else:
            s.append(0)
            
    return s

In [45]:
df["ERI"] = strategy_ERI(P_close=df.Close, P_high=df.High, P_low=df.Low, n=13)

In [46]:
df.head()

,Open,High,Low,Close,Volume,ERI,DI,DI_,BeP_,BuP_
Date,,,,,,,,,,
2000-01-31,1360.16,1394.48,1350.14,1394.46,993800000,NaN,NaN,NaN,-44.320000,0.020000
2000-02-01,1394.46,1412.49,1384.79,1409.28,981000000,0.0,NaN,NaN,-11.787143,15.912857
2000-02-02,1409.28,1420.61,1403.49,1409.12,1038600000,1.0,NaN,NaN,5.121020,22.241020
2000-02-03,1409.12,1425.78,1398.52,1424.97,1146500000,0.0,NaN,NaN,-3.649125,23.610875
2000-02-04,1424.97,1435.91,1420.63,1424.37,1045100000,1.0,0.0,0.000084,15.289321,30.569321


In [47]:
df["DI"] = strategy_DI(P_close=df.Close, n=5)
df.head()

,Open,High,Low,Close,Volume,ERI,DI,DI_,BeP_,BuP_
Date,,,,,,,,,,
2000-01-31,1360.16,1394.48,1350.14,1394.46,993800000,NaN,NaN,NaN,-44.320000,0.020000
2000-02-01,1394.46,1412.49,1384.79,1409.28,981000000,0.0,NaN,NaN,-11.787143,15.912857
2000-02-02,1409.28,1420.61,1403.49,1409.12,1038600000,1.0,NaN,NaN,5.121020,22.241020
2000-02-03,1409.12,1425.78,1398.52,1424.97,1146500000,0.0,NaN,NaN,-3.649125,23.610875
2000-02-04,1424.97,1435.91,1420.63,1424.37,1045100000,1.0,0.0,0.000084,15.289321,30.569321


In [48]:
df.tail()

,Open,High,Low,Close,Volume,ERI,DI,DI_,BeP_,BuP_
Date,,,,,,,,,,
2021-03-05,3793.58,3851.69,3730.19,3841.94,6842570000,0.0,1.0,0.000004,-123.019243,-1.519243
2021-03-08,3844.39,3881.06,3819.25,3821.35,5852240000,0.0,-1.0,-0.000008,-29.407922,32.402078
2021-03-09,3851.93,3903.76,3851.93,3875.44,5496340000,0.0,1.0,0.000131,-0.553934,51.276066
2021-03-10,3891.99,3917.35,3885.73,3898.81,5827250000,1.0,0.0,0.000150,26.628057,58.248057
2021-03-11,3915.54,3960.27,3915.54,3939.34,5300010000,0.0,0.0,0.000165,44.975477,89.705477


In [49]:
df["DI_"] = DI(df.Close, n=5, MA_type="SMA")
df["BeP_"] = ERI(df.Close, df.High, df.Low, n=13)[1]
df["BuP_"] = ERI(df.Close, df.High, df.Low, n=13)[0]
df.head()

,Open,High,Low,Close,Volume,ERI,DI,DI_,BeP_,BuP_
Date,,,,,,,,,,
2000-01-31,1360.16,1394.48,1350.14,1394.46,993800000,NaN,NaN,NaN,-44.320000,0.020000
2000-02-01,1394.46,1412.49,1384.79,1409.28,981000000,0.0,NaN,NaN,-11.787143,15.912857
2000-02-02,1409.28,1420.61,1403.49,1409.12,1038600000,1.0,NaN,NaN,5.121020,22.241020
2000-02-03,1409.12,1425.78,1398.52,1424.97,1146500000,0.0,NaN,NaN,-3.649125,23.610875
2000-02-04,1424.97,1435.91,1420.63,1424.37,1045100000,1.0,0.0,0.000084,15.289321,30.569321


In [50]:
df.to_excel('d:\Курсова\Курсова 2021\check 5.xlsx')